In [3]:
library(magrittr)
library(dplyr)
library(glue)
library(data.table)
library(ggplot2)
library(tfdatasets)
library(tensorflow)
library(keras)
tf$version$VERSION
## load data Channel Dataset-----------------------------------------------------
load(file = 'df_paths.rdata',verbose = T)

[1] "2.1.0"

Loading objects:
  df_paths
  train_paths
  test_paths


In [4]:
ft_spec_target <- df_paths %>%
      select(-customer_id,-path_no,-path) %>% 
      feature_spec(target ~ .) %>%
      step_categorical_column_with_vocabulary_list(starts_with("V"),vocabulary_list = list('channel_0', 'channel_1', 'channel_2', 'channel_3', 'channel_4', 'channel_5', 'channel_6', 'channel_7', 'channel_8')) %>%
      step_embedding_column(starts_with("V"), dimension = function(vocab_size) as.integer(sqrt(vocab_size) + 1)   ## dim=4
      ) %>%
  fit()

ft_spec_target$dense_features() 

$embedding_V1
EmbeddingColumn(categorical_column=VocabularyListCategoricalColumn(key='V1', vocabulary_list=('channel_0', 'channel_1', 'channel_2', 'channel_3', 'channel_4', 'channel_5', 'channel_6', 'channel_7', 'channel_8'), dtype=tf.string, default_value=-1, num_oov_buckets=0), dimension=4, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True)

$embedding_V2
EmbeddingColumn(categorical_column=VocabularyListCategoricalColumn(key='V2', vocabulary_list=('channel_0', 'channel_1', 'channel_2', 'channel_3', 'channel_4', 'channel_5', 'channel_6', 'channel_7', 'channel_8'), dtype=tf.string, default_value=-1, num_oov_buckets=0), dimension=4, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True)

$embedding_V3
EmbeddingColumn(categorical_column=VocabularyListCategoricalColumn(key='V3',

In [6]:
cat('Build model...\n')
 n_lstm = 7
batch_size = 32

rnn_in <- layer_input_from_dataset(df_paths %>% select(-target)) 

rnn_out <- 
  rnn_in %>%
  layer_dense_features(ft_spec_target$dense_features()) %>%
  layer_reshape(target_shape= list(5,4))  %>%
  bidirectional( layer_lstm(units = n_lstm, dropout = 0.2, recurrent_dropout = 0.2) ) %>%  # bidirectional not gain
  layer_dense(units = 1, activation = 'sigmoid')

rnn <- keras_model(rnn_in, rnn_out)

rnn %>% 
  compile(
    loss = "binary_crossentropy", 
    optimizer = "adam",
    metrics = "accuracy"
  )

rnn %>% fit(x = df_paths %>% select(-target), y = df_paths$target, verbose=2,
            validation_split = 0.2, epoch=20 ,batch_size=batch_size )

Build model...


In [1]:
rnn_scored = cbind( df_paths , rnn_pred = predict(rnn, df_paths) ) #%$% table(target,rnn_pred.V1)
rnn_scored[customer_id=='id1000']

## alter a value 

df_altered1 = df_paths[customer_id=='id1000', V4 := 'channel_5'][customer_id=='id1000'] 
df_altered1
predict(rnn, df_altered1 )  ## Change in pred

df_altered2 = df_paths[customer_id=='id1000', V4 := 'channel_0'][customer_id=='id1000'] 
predict(rnn, df_altered2 )  ## Change in pred

ERROR: Error in cbind(df_paths, rnn_pred = predict(rnn, df_paths)): object 'df_paths' not found
